# RAISE the Bar: Restriction of Action Spaces for Improved Social Welfare and Equity in Traffic Management
## Random Erdős-Rényi graph experiment

### Setup and function definitions

In [ ]:
# External modules
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import statsmodels.formula.api as sm
from scipy import interpolate

# Own modules
import os, sys
sys.path.append(f'{os.getcwd()}/../../')

from src.environment import TrafficModel, Car, create_cars, build_network, UniformLatencyGenerator
from src.analysis import analyze_fairness
from src.util import change_value_of_money

plt.rcParams['text.usetex'] = True

In [ ]:
def create_random_gnp_graph(number_of_nodes, p, latency_generator, *, seed=42):
    network = nx.gnp_random_graph(number_of_nodes, p, seed=seed, directed=True)

    nx.set_node_attributes(
        network,
        nx.spring_layout(network, k=1.75, seed=103),
        "position",
    )

    nx.set_edge_attributes(
        network,
        {edge: latency_generator() for edge in network.edges},
        "latency_params",
    )

    return build_network(network)

In [ ]:
def interpolate_travel_times(car_travel_times):
    interpolated_car_stats = pd.DataFrame()
    for seed in car_travel_times.seed.unique():
        grouped_seed_car_stats = car_travel_times[car_travel_times['seed'] == seed].groupby('step').mean().reset_index()

        interpolation_function = interpolate.interp1d(grouped_seed_car_stats['step'],
                                                      grouped_seed_car_stats['travel_time'],
                                                      fill_value="extrapolate")

        interpolation_results = pd.DataFrame({'step': range(1, 10001)})
        interpolation_results.set_index('step')
        interpolation_results['travel_time'] = interpolation_results.apply(lambda x: interpolation_function(x.index))
        interpolation_results['seed'] = seed
        interpolated_car_stats = pd.concat([interpolated_car_stats, interpolation_results])

    return interpolated_car_stats

In [ ]:
BASE_COLOR = 'aqua'
RESTRICTION_COLOR = 'coral'
TOLLING_COLOR = 'darkred'

car_seeds = [41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
number_of_steps = 10000
values_of_money = [0, 1, 2, 5]

# Setup 1
# number_of_nodes = 50
# p = 0.070711
# car_counts = {(23, 33): 38}
# c_max = 3
# c_min = 1
# b_max = 3
# b_min = 2
# a_max = 4
# a_min = 3
# restriction_edge = (23, 30)
# R = 7.4e-4
# beta = 1

# Setup 2
# number_of_nodes = 34
# p = 0.058310
# car_counts = {(6, 20): 38}
# c_max = 4
# c_min = 1
# b_max = 4
# b_min = 2
# a_max = 4
# a_min = 3
# restriction_edge = (25, 9)
# R = 0.9
# beta = 1

# Setup 3
number_of_nodes = 59
p = 0.076811
car_counts = {(3, 27): 31}
c_max = 11
c_min = 3
b_max = 3
b_min = 2
a_max = 4
a_min = 2
restriction_edge = (20, 41)
R = 0.000050
beta = 1

### Network Graph

In [ ]:
network = create_random_gnp_graph(number_of_nodes=number_of_nodes, p=p,
                                      latency_generator=UniformLatencyGenerator(a_min=a_min, a_max=a_max,
                                                                                b_min=b_min, b_max=b_max,
                                                                                c_min=c_min, c_max=c_max),
                                      seed=46)

color_map = []
edge_colors = []
labels = {}
node_sizes = []
for i, node in enumerate(network):
    if node == 27:
        color_map.append('#D5E8D4')
        edge_colors.append('#82B366')
        node_sizes.append(250)
        labels[node] = r"$t$"
    elif node == 3:
        color_map.append('#DAE8FC')
        edge_colors.append('#6C8EBF')
        node_sizes.append(250)
        labels[node] = r"$s$"
    else:
        color_map.append('#F5F5F5')
        edge_colors.append('#666666')
        node_sizes.append(175)
        labels[node] = fr"$s_{{{i}}}$"

fig, ax = plt.subplots()
ax.figure.set_size_inches(5, 5)
nx.draw(
    network,
    ax=ax,
    pos=nx.get_node_attributes(network, "position"),
    labels=labels,
    font_size=6,
    edgelist=[(v, w) for v, w in network.edges if v != w],
    node_size=node_sizes,
    width=0.7,
    node_color=color_map,
    edgecolors=edge_colors
)

plt.tight_layout()
ax.get_figure().savefig('network.pdf', dpi=300)

### Experiment

In [ ]:
results = []
unrestricted_car_travel_times = pd.DataFrame(columns=['step', 'seed', 'travel_time'])
restricted_car_travel_times = pd.DataFrame(columns=['step', 'seed', 'travel_time', 'improvement'])
tolling_car_travel_times = pd.DataFrame(columns=['step', 'seed', 'travel_time', 'improvement'])
total_cost_car_travel_times = pd.DataFrame(columns=['step', 'seed', 'travel_time'])
for seed in car_seeds:
    print(f'Running with seed {seed}.')
    # Unrestricted
    network = create_random_gnp_graph(number_of_nodes=number_of_nodes, p=p,
                                      latency_generator=UniformLatencyGenerator(a_min=a_min, a_max=a_max,
                                                                                b_min=b_min, b_max=b_max,
                                                                                c_min=c_min, c_max=c_max),
                                      seed=46)
    cars = create_cars(network, car_counts=car_counts, seed=seed)
    change_value_of_money(cars, values_of_money, seed=seed)
    model = TrafficModel(network, cars, seed=seed)
    step_stats_unrestricted, car_stats_unrestricted = model.run_sequentially(number_of_steps, show_progress=False)

    grouped_car_stats = car_stats_unrestricted[['step', 'travel_time']]\
        .groupby('step').mean().expanding().mean().reset_index()
    grouped_car_stats['seed'] = seed
    unrestricted_car_travel_times = pd.concat([unrestricted_car_travel_times, grouped_car_stats])

    results.append({'seed': seed,
                    'scenario': 'Base',
                    'travel_time': car_stats_unrestricted["travel_time"][-5000:].mean(),
                    **analyze_fairness(car_stats_unrestricted),
                    **{index: value for index, value in car_stats_unrestricted[-5000:]
                   .groupby('value_of_money')['travel_time'].mean().items()}})

    # Restricted
    network = create_random_gnp_graph(number_of_nodes=number_of_nodes, p=p,
                                      latency_generator=UniformLatencyGenerator(a_min=a_min, a_max=a_max,
                                                                                b_min=b_min, b_max=b_max,
                                                                                c_min=c_min, c_max=c_max),
                                      seed=46)
    cars = create_cars(network, car_counts=car_counts, seed=seed)
    model = TrafficModel(network, cars, seed=seed)

    model.set_edge_restriction(restriction_edge, allowed=False)
    model.cars = create_cars(network, car_counts=car_counts, seed=seed)
    change_value_of_money(model.cars, values_of_money, seed=seed)

    step_stats_restricted, car_stats_restricted = model.run_sequentially(number_of_steps, show_progress=False)

    grouped_car_stats = car_stats_restricted[['step', 'travel_time']]\
        .groupby('step').mean().expanding().mean().reset_index()
    grouped_car_stats['seed'] = seed
    restricted_car_travel_times = pd.concat([restricted_car_travel_times, grouped_car_stats])

    results.append({'seed': seed,
                    'scenario': 'Restricted',
                    'travel_time': car_stats_restricted["travel_time"][-5000:].mean(),
                    **analyze_fairness(car_stats_restricted),
                    **{index: value for index, value in car_stats_restricted[-5000:]
                   .groupby('value_of_money')['travel_time'].mean().items()}})

    # Tolling
    network = create_random_gnp_graph(number_of_nodes=number_of_nodes, p=p,
                                      latency_generator=UniformLatencyGenerator(a_min=a_min, a_max=a_max,
                                                                                b_min=b_min, b_max=b_max,
                                                                                c_min=c_min, c_max=c_max),
                                      seed=46)
    cars = create_cars(network, car_counts=car_counts, seed=seed)
    change_value_of_money(cars, values_of_money, seed=seed)
    model = TrafficModel(network, cars, tolls=True, beta=beta, R=R, seed=seed)
    step_stats_tolling, car_stats_tolling = model.run_sequentially(number_of_steps, show_progress=False)

    grouped_car_stats = car_stats_tolling[['step', 'travel_time']]\
    .groupby('step').mean().expanding().mean().reset_index()
    grouped_car_stats['seed'] = seed
    tolling_car_travel_times = pd.concat([tolling_car_travel_times, grouped_car_stats])

    results.append({'seed': seed,
                    'scenario': 'Tolling (excl. tolls)',
                    'travel_time': car_stats_tolling["travel_time"][-5000:].mean(),
                    **analyze_fairness(car_stats_tolling),
                    **{index: value for index, value in car_stats_tolling[-5000:]
                   .groupby('value_of_money')['travel_time'].mean().items()}})

    grouped_car_stats = car_stats_tolling[['step', 'total_cost']]\
    .groupby('step').mean().expanding().mean().reset_index()
    grouped_car_stats.columns = ['step', 'travel_time']

    grouped_car_stats['seed'] = seed
    total_cost_car_travel_times = pd.concat([total_cost_car_travel_times, grouped_car_stats])

    results.append({'seed': seed,
                    'scenario': 'Tolling (incl. tolls)',
                    'travel_time': car_stats_restricted["total_cost"][-5000:].mean(),
                    **analyze_fairness(car_stats_restricted),
                    **{index: value for index, value in car_stats_restricted[-5000:]
                   .groupby('value_of_money')['total_cost'].mean().items()}})

results = pd.DataFrame(results).set_index(['scenario'])
results = results.reset_index()

print('Interpolating travel times...')
unrestricted_car_travel_times = interpolate_travel_times(unrestricted_car_travel_times)
restricted_car_travel_times = interpolate_travel_times(restricted_car_travel_times)
tolling_car_travel_times = interpolate_travel_times(tolling_car_travel_times)
total_cost_car_travel_times = interpolate_travel_times(total_cost_car_travel_times)
print('DONE!')

In [ ]:
df = results[['scenario', 0, 1, 2, 5]].groupby('scenario')
df_mean = df.mean().drop(['Tolling (incl. tolls)'], axis='index')
df_std = df.std().drop(['Tolling (incl. tolls)'], axis='index')

ax = df_mean.plot.bar(cmap=mpl.colormaps['winter'], capsize=4, yerr=df_std)
ax.legend(loc='lower left', title='Value of money')
plt.xticks(rotation = 0)
ax.set_ylabel('Travel time')
ax.set_xlabel(None)

for offset, scenario in enumerate(results['scenario'].unique()):
    if scenario == 'Tolling (incl. tolls)':
        break

    test = results[['scenario', 'seed', 0, 1, 2, 5]].groupby(['scenario', 'seed']).mean().reset_index()
    test = test[test['scenario'] == scenario].drop(['scenario', 'seed'], axis=1).melt(var_name="value_of_money",
                                                                                    value_name="travel_time")
    test = test.astype(float)
    ols = sm.ols(formula='travel_time ~ value_of_money', data=test).fit()
    print(f'{scenario} p-value: {ols.pvalues["value_of_money"]}')
    slope = ols.params['value_of_money']
    intercept = ols.params['Intercept']
    ax.plot([offset - 0.3, offset + 0.3],
        [intercept + slope * min(values_of_money) + 2.0,
         intercept + slope * max(values_of_money) + 2.0],
        'r--')

    ax.text(offset, ((intercept + slope * min(values_of_money) + 2.0)
                     + (intercept + slope * max(values_of_money) + 2.0)) / 2 + 0.6,
            str(np.round(slope, 2)) if np.round(slope, 2) != -0.0 else '0.0',
            horizontalalignment='center', fontsize=10, fontweight=100)

    if offset == 2:
        ax.text(2, ((intercept + slope * min(values_of_money) + 2.0)
                         + (intercept + slope * max(values_of_money) + 2.0)) / 2 + 1.0,
                '***',
                horizontalalignment='center', fontsize=8)

plt.tight_layout()
ax.get_figure().savefig('fairness.pdf', dpi=300)

In [ ]:
mean = unrestricted_car_travel_times.groupby('step')['travel_time'].mean()
std = unrestricted_car_travel_times.groupby('step')['travel_time'].std()
ax = mean.plot(xlabel='step', label='Unrestricted')
ax.fill_between(std.index,
                mean - std,
                mean + std,
                alpha=0.25)

mean = restricted_car_travel_times.groupby('step')['travel_time'].mean()
std = restricted_car_travel_times.groupby('step')['travel_time'].std()
ax = mean.plot(xlabel='step', label='Restricted')
ax.fill_between(std.index,
                mean - std,
                mean + std,
                alpha=0.25)

mean = tolling_car_travel_times.groupby('step')['travel_time'].mean()
std = tolling_car_travel_times.groupby('step')['travel_time'].std()
ax = mean.plot(xlabel='step', label='Tolling (excl. tolls)')
ax.fill_between(std.index,
                mean - std,
                mean + std,
                alpha=0.25)

ax.legend(loc='lower right')
ax.set_xlabel('Step')
ax.set_ylabel('Mean travel time')
ax.set_ylim([16, 18])
ax.get_xaxis().set_major_formatter(
    mpl.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

plt.tight_layout()
ax.get_figure().savefig('absolute-travel-time.pdf', dpi=300)

In [ ]:
improvements =  unrestricted_car_travel_times.groupby(['step', 'seed'])['travel_time'].mean().div(
    restricted_car_travel_times.groupby(['step', 'seed'])['travel_time'].mean()).sub(1.0).reset_index().groupby('step')
mean = improvements.mean()['travel_time']
std = improvements.std()['travel_time']
ax = mean.plot(label='Restricted')
ax.fill_between(std.index,
                mean - std,
                mean + std,
                alpha=0.25)

improvements =  unrestricted_car_travel_times.groupby(['step', 'seed'])['travel_time'].mean().div(
    tolling_car_travel_times.groupby(['step', 'seed'])['travel_time'].mean()).sub(1.0).reset_index().groupby('step')
mean = improvements.mean()['travel_time']
std = improvements.std()['travel_time']
ax = mean.plot(label='Tolling (excl. tolls)')
ax.fill_between(std.index,
                mean - std,
                mean + std,
                alpha=0.25)

ax.legend(loc='lower right')
ax.set_xlabel('Step')
ax.set_ylabel('Improvement relative to \emph{Base}')
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.axhline(ls='--', color='grey')
ax.get_xaxis().set_major_formatter(
    mpl.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

plt.tight_layout()
ax.get_figure().savefig('relative-travel-time.pdf', dpi=300)